In [16]:
from models import train_predict,CondensedGoogLeNet
from models.train_predict import get_confusion_matrix
from data.data_processing import generate_dummy_data,DataHandler,get_training_data_from_path
from config.hyperparams import GoogleNet_hyperparams
from backtest.backtest import Backtester


import tensorflow as tf
import pandas as pd
import numpy as np
import os

%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
folder_for_samples = 'data/cnn_samples'

# Data Processing

## Get stock data in Handler object

In [12]:
# path_to_dump_samples = os.path.join(folder_for_samples,'regular')
path_to_dump_samples = os.path.join(folder_for_samples,'close-1_week-2015_2018-64_pixels-high_VOL')

handler = DataHandler(encoding_method='GADF', window_len=64, image_size=64, retrain_freq=5,
                 start_date = 20150101, end_date= 20200102, frac_of_stocks=1.,
                stock_data_dir_path= 'data/2019_stock_data',targets_methods=['close'],
                 dir_for_samples=path_to_dump_samples, nb_of_stocks_by_file=100,minimum_volume=8e6,
                      kwargs_target_methods = {'up_return':0.013,'down_return':-0.012,'buy_on_last_date':True}
                 )

[autoreload of backtest.backtest failed: Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\extensions\autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\extensions\autoreload.py", line 368, in superreload
    module = reload(module)
  File "C:\ProgramData\Anaconda3\lib\imp.py", line 315, in reload
    return importlib.reload(module)
  File "C:\ProgramData\Anaconda3\lib\importlib\__init__.py", line 166, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 618, in _exec
  File "<frozen importlib._bootstrap_external>", line 674, in exec_module
  File "<frozen importlib._bootstrap_external>", line 781, in get_code
  File "<frozen importlib._bootstrap_external>", line 741, in source_to_code
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "C:\Users\elior illouz\Notebooks\Deep Learning\Dee

In [13]:
handler.get_df_data()

Textarea(value='', layout=Layout(border='1px solid #999999', height='200px', width='100%'))

In [8]:
# handler.show_image(handler.df_data)

## Builds images and targets and creates pickles
+ Be careful for the folder name (specified in handler object), this function will delete all files in this folder before dumping pickles

In [9]:
handler.build_and_dump_images_and_targets()

Textarea(value='', layout=Layout(border='1px solid #999999', height='200px', width='100%'))

C:\Users\elior illouz\Notebooks\Deep Learning\DeepLearningAllocationCNN\data\data_processing.py:314: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  df_res = pd.concat([df_res, df_res_one_permno])


# Training

## Retrieve data from the specified folder
+ '/regular' is the default to retrieve samples
+ '/VWAP_1_week_2015_2017_all_stocks' is another one

In [3]:
%%time

# path_to_retrieve_samples = folder_for_samples+'/regular'
path_to_retrieve_samples = folder_for_samples+'/close-1_week-2015_2018-64_pixels-high_VOL'



X_train, X_val, X_test, Y_train, Y_val, Y_test = get_training_data_from_path(samples_path=path_to_retrieve_samples,
                                targets_type='close',
                                train_val_size=2 / 3.,
                                train_size=0.75,
                                logger_env='Training')


Textarea(value='', layout=Layout(border='1px solid #999999', height='200px', width='100%'))

Wall time: 8.45 s


## Build the neural network object and Train the network 
+ Choose your params BE CAREFUL about the path to drop the model_checkpoints
+ Choose or update hyperparameters
+ If you run restore = True Choose the checkpoint.meta file to use for the model to load

###### Then 

+ Choose parameters for the train_predict function 
+ Run the next cell to build the network object and train

In [4]:
shape_x,shape_y = list(X_train.shape),list(Y_train.shape)
shape_x[0],shape_y[0] = None,None

GoogleNet_hyperparams.update({'learning_rate':0.003,'dropout':0.1})

# penalty factor assigned for misclassifying i to j. It used to combine sample prior (distribution of classes in the training
# sample) to compute expected penalty of misclassifying i-th class
cost_matrix=np.array([[0, 1.1, 1.4], [1.1, 0, 1.1], [1.4, 1.1, 0]])

n_test = 5000
checkpoints_folder = 'model_checkpoints'
checkpoint_prefix = 'ckpt-close-1_week-2015_2018-64_pixels-high_VOL'
model_checkpoints_path = os.path.join(checkpoints_folder,checkpoint_prefix)

# X_train1, X_val1, Y_train1, Y_val1 =  X_train[:n_test], X_val[:n_test], Y_train[:n_test], Y_val[:n_test]
# X_train1, X_val1, Y_train1, Y_val1 =  X_train[n_test:], X_val[:n_test], Y_train[n_test:], Y_val[:n_test]
X_train1, X_val1, Y_train1, Y_val1 =  X_train, X_val, Y_train, Y_val



path_restore = (tf.train.latest_checkpoint('model_checkpoints') or '')+'.meta'

#### Run the training

In [5]:
network = CondensedGoogLeNet(shape_x=shape_x,shape_y=shape_y,hyperparams=GoogleNet_hyperparams)


GN_pred, GN_training_loss, GN_val_loss = train_predict(X=X_train1, Y=Y_train1, batch_size=20, valX=X_val1, valY=Y_val1,
                                                       model_ckpt_path=model_checkpoints_path, is_bayesian=True,cost_matrix=cost_matrix,
                                                       epochs=5, net=network,save_step =2,cost_sensitive_loss=True,restore=False,
                                                       model_ckpt_path_to_restore=path_restore)

Textarea(value='', layout=Layout(border='1px solid #999999', height='200px', width='100%'))

KeyboardInterrupt: 

In [ ]:
pd.DataFrame([GN_training_loss,GN_val_loss]).T.plot()

# Backtest

In [47]:
checkpoints_folder = 'model_checkpoints'
checkpoint_prefix = 'ckpt-close-1_week-2015_2018-64_pixels-high_VOL'
model_checkpoints_path = os.path.join(checkpoints_folder,checkpoint_prefix)

path_to_retrieve_samples = folder_for_samples+'/close-1_week-2015_2018-64_pixels-high_VOL'


model_checkpoints_path_file = model_checkpoints_path+'-840.meta'

backtester = Backtester(path_data=path_to_retrieve_samples,path_model_to_restore=model_checkpoints_path_file,start_date=20160406,end_date=20200609)

In [48]:
backtester.run_backtest()

Textarea(value='', layout=Layout(border='1px solid #999999', height='200px', width='100%'))

INFO:tensorflow:Restoring parameters from model_checkpoints\ckpt-close-1_week-2015_2018-64_pixels-high_VOL-840


In [13]:
backtester._df_permnos_to_buy.loc[20180125:20180205]

,10_max_long,20_max_long,2_max_long
date,,,
20180129,"[86580.0, 85442.0, 91232.0, 22752.0, 86456.0, ...","[86580.0, 85442.0, 91232.0, 22752.0, 86456.0, ...","[86580.0, 85442.0]"
20180205,"[63765.0, 12060.0, 59328.0, 53613.0, 85442.0, ...","[63765.0, 12060.0, 59328.0, 53613.0, 85442.0, ...","[63765.0, 12060.0]"


In [14]:
# file_names = ['stockdata_{}'.format(i) for i in range(1,18)]
# df_data = handler._load_stock_data(file_names, data_dir_path=handler._stock_data_dir_path,
#                                         logger_env=handler._LOGGER_ENV)

In [15]:
# df_data_ret = df_data.copy()

In [16]:
# df_data_ret = df_data_ret[['date','PRC']].pivot_table(index='date',columns='PERMNO')
# df_data_ret = np.abs(df_data_ret['PRC'])
# df_logs = np.abs(np.log(df_data_ret.shift(1)/df_data_ret))


In [17]:
# booleans  =(df_logs>np.log(2.))
# permno =[]
# for col in booleans.columns:
#     if np.any(booleans[col]):
#         permno.append(col)

In [133]:
set(df_data.loc[permno].COMNAM)

{'3 D SYSTEMS CORP DEL',
 '51JOB INC',
 'A C I WORLDWIDE INC',
 'A N I PHARMACEUTICALS INC',
 'A S A GOLD & PRECIOUS METALS LTD',
 'A S A LIMITED',
 'A T A INC',
 'A Z Z INC',
 'AASTROM BIOSCIENCES INC',
 'ABBOTT LABORATORIES',
 'ABERDEEN GLOBAL DYNAMIC DIV FD',
 'ABERDEEN TOTAL DYNAMIC DIV FD',
 'ACCELERATE DIAGNOSTICS INC',
 'ACCELR8 TECHNOLOGY CORP',
 'ACER THERAPEUTICS INC',
 'ACETO CORP',
 'ACHIEVE LIFE SCIENCES INC',
 'ACHILLION PHARMACEUTICALS INC',
 'ACORN INTERNATIONAL INC',
 'ADCARE HEALTH SYSTEMS INC',
 'ADEONA PHARMACEUTICALS INC',
 'ADVENTRX PHARMACEUTICALS INC',
 'AETERNA ZENTARIS INC',
 'AFLAC INC',
 'AGENUS INC',
 'AKORN INC',
 'ALBIREO PHARMA INC',
 'ALCOA INC',
 'ALEXION PHARMACEUTICALS INC',
 'ALLIANCE ONE INTERNATIONAL INC',
 'ALLIED HEALTHCARE PRODUCTS INC',
 'ALPHATEC HOLDINGS INC',
 'ALPINE GLOBAL DYNAMIC DIVID FD',
 'ALPINE TOTAL DYNAMIC DIVIDEND FD',
 'ALTIMMUNE INC',
 'AMERICAN ELECTRIC TECHNOLS INC',
 'AMERICAN LORAIN CORPORATION',
 'AMERICAN SUPERCONDUCTOR C

In [129]:
permno

[10180,
 10253,
 10258,
 10333,
 10443,
 10550,
 10656,
 10696,
 10838,
 10892,
 11018,
 11308,
 11394,
 11481,
 11618,
 11628,
 11644,
 11762,
 11809,
 12266,
 14593,
 16126,
 16505,
 16678,
 17144,
 18148,
 19166,
 20053,
 20482,
 20598,
 22323,
 22509,
 23393,
 23536,
 23887,
 24643,
 25232,
 26614,
 26649,
 27959,
 27983,
 28118,
 29867,
 30737,
 30940,
 31051,
 32803,
 34666,
 37381,
 37568,
 37584,
 38295,
 38659,
 38762,
 39571,
 43553,
 44813,
 45225,
 47677,
 51263,
 51692,
 52337,
 53373,
 55001,
 55212,
 55634,
 57293,
 57568,
 57665,
 57809,
 57904,
 58836,
 58975,
 59483,
 59504,
 61815,
 62033,
 62148,
 62156,
 63263,
 63467,
 63546,
 64450,
 64629,
 64822,
 64961,
 65402,
 66325,
 66835,
 69586,
 70703,
 74500,
 75047,
 75261,
 75470,
 75672,
 75844,
 75905,
 76082,
 76215,
 76221,
 76230,
 76261,
 76360,
 76582,
 76613,
 76804,
 76858,
 76932,
 76963,
 77175,
 77202,
 77236,
 77259,
 77274,
 77437,
 77486,
 77501,
 77699,
 78002,
 78015,
 78044,
 78071,
 78081,
 78156,


In [77]:
set(df_data.loc[91387].TICKER)

{'SDS'}

In [24]:
backtester._df_all_data[backtester._df_all_data.PERMNO==63765.0].loc[20180125:20180205]

,PERMNO,PRC,long,hold,short
date,,,,,
20180129,63765.0,4.75,0.585555,0.143118,0.271328
20180205,63765.0,3.85,0.517942,0.275125,0.206932


In [51]:
tmp = backtester.plot_backtest()

In [45]:
backtester.df_strats

,10_max_long,20_max_long,2_max_long,Cash
date,,,,
2015-04-13,1.004381,1.010277,1.071072,1.000000
2015-04-20,1.053073,1.045204,1.103819,1.003809
2015-04-27,1.045969,1.051027,1.084050,1.007881
2015-05-04,1.062162,1.083221,1.109006,1.010543
2015-05-11,1.073787,1.081304,1.134837,1.006165
2015-05-18,1.091121,1.099712,1.149497,1.017573
2015-05-26,1.065991,1.082422,1.143455,1.005625
2015-06-02,1.074504,1.093968,1.173525,1.008206
2015-06-09,1.057340,1.075251,1.112863,0.994131


In [52]:
tmp

# Little tests section

In [ ]:
import tensorflow as tf

In [ ]:
tf.train.latest_checkpoint('model_checkpoints')

In [ ]:

# with tf.variable_scope('test',reuse=tf.AUTO_REUSE):
#     network2 = CondensedGoogLeNet(shape_x=shape_x,shape_y=shape_y,hyperparams=GoogleNet_hyperparams)
#     network2.build_operations()
sess=tf.Session()    
#First let's load meta graph and restore weights
# saver = tf.train.import_meta_graph('model_checkpoints/ckpt1-266.meta')
saver = tf.train.import_meta_graph('model_checkpoints/ckpt-26.meta')
saver.restore(sess,tf.train.latest_checkpoint('model_checkpoints'))

In [ ]:
graph = tf.get_default_graph()
# w1 = graph.get_tensor_by_name("w1:0")
# w2 = graph.get_tensor_by_name("w2:0")
# feed_dict ={w1:13.0,w2:17.0}



+ CondensedGoogLeNet/loss/Mean gets the loss
+ CondensedGoogLeNet/Adam gets the optimizer
+ CondensedGoogLeNet/accuracy/Mean gets the accuracy
    + we can get the validation error just after
+ CondensedGoogLeNet/Merge/MergeSummary gets the summary op
+ CondensedGoogLeNet/init gets init_op
+ 

In [ ]:
output = graph.get_tensor_by_name('CondensedGoogLeNet/output:0')
train_op = graph.get_tensor_by_name('CondensedGoogLeNet/minimize:0')
x = graph.get_tensor_by_name('CondensedGoogLeNet/x:0')
y = graph.get_tensor_by_name('CondensedGoogLeNet/y:0')
global_step = graph.get_tensor_by_name('CondensedGoogLeNet/global_step:0')
# loss = graph.get_tensor_by_name('CondensedGoogLeNet/loss/Mean:0')

In [ ]:
pred = sess.run([train_op,global_step],feed_dict={x:X_val[:100],y:Y_val[:100]})

In [ ]:
pred

In [ ]:
network.optimizer

In [ ]:
# train_op = graph.get_tensor_by_name('CondensedGoogLeNet/minimize:0')


In [ ]:
train_op

In [ ]:
pred

In [ ]:
pred = sess.run(output,feed_dict={x:X_val1[:15]})
pred

In [ ]:
saver.save(sess,'model_checkpoints/test_restore')

In [ ]:
pd.DataFrame(np.max(pred,axis=1)).hist(bins=100)

In [ ]:
pred[:5]

In [ ]:
GN_pred[:5]

In [ ]:
Y_val[:5]

In [ ]:
pred.shape

In [ ]:
tf.reset_default_graph()

# Create some variables.
v1 = tf.get_variable("v1", shape=[3])
v2 = tf.get_variable("v2", shape=[5])

# Add ops to save and restore all the variables.
saver = tf.train.Saver()

# Later, launch the model, use the saver to restore variables from disk, and
# do some work with the model.
with tf.Session() as sess:
  # Restore variables from disk.
  saver.restore(sess, "/tmp/model.ckpt")
  print("Model restored.")
  # Check the values of the variables
  print("v1 : %s" % v1.eval())
  print("v2 : %s" % v2.eval())

In [ ]:
tf.train.latest_checkpoint

In [ ]:
tf.train.import_meta_graph